In [1]:
#importing tensorflow
import tensorflow as tf
print(tf.__version__)

2.0.0


In [2]:
#importing tensorflow data services which holds a corpus of several datesets
import tensorflow_datasets as tfds 
#loading the imdb_reviews dataset which has 50,000 movie reviews, labeled either as positive or negative
#the dataset has been uploaded in the repository for reference
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True) 

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [41]:
#the data is splitted into 25,000 reviews for training and 25,000 reviews for testing
import numpy as np

train_data, test_data = imdb['train'], imdb['test'] #splitting the training and testing/validation data

training_sentences = [] #list for training sentences
training_labels = [] #list for training labels (1:Positive review, 0:Negative review)

testing_sentences = [] #list for testing sentences
testing_labels = [] #list fro testing labels (1:Postive review, 0:Negative review)

#values for s and l are tensors so calling s.numpy() and l.numpy() will extract thier values
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8')) 
  training_labels.append(l.numpy())

#values for s and l are tensors so calling s.numpy() and l.numpy() will extract thier values
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

print(training_sentences[0],end='\n')
print(training_labels_final[0])
#as visible the review is negative, hence the corresponding label too is negative i.e 0

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
0


In [42]:
#here we will tokenize and pad the sentences

vocab_size = 10000 #the maximum number of words to keep, based on word frequency
embedding_dim = 16 #dimension for the word embedding vectors
max_length = 120 #keeping the lengths of all sentences the same as 120
trunc_type='post' #truncating from the end of the sentences is sentences length is more than 120
oov_tok = "<OOV>" #designating the out of vocab token as "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok) #creating the tokenizer object
tokenizer.fit_on_texts(training_sentences) #generating tokens by fitting on the training_sentences_list
word_index = tokenizer.word_index #generating a dictionary of words as key and the value as their tokens
sequences = tokenizer.texts_to_sequences(training_sentences) #generating the sequence of texts as their corresponding tokens
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type) #padding the sequences to make them all of the same length

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [45]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) #reversing the word_index

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), #adding the embedding layer which has the shape of 120x16
    tf.keras.layers.Flatten(), #the values of embedding layers are flattened here (120*16=1920)
    tf.keras.layers.Dense(6, activation='relu'), #adding dense layer of 6 units
    tf.keras.layers.Dense(1, activation='sigmoid') #adding one output unit in the layer with a sigmoid activation as we are doing binary classification
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) #compiling the model
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [48]:
num_epochs = 10 
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5089 - accuracy: 0.7230 - val_loss: 0.3498 - val_accuracy: 0.8470
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2447 - accuracy: 0.9059 - val_loss: 0.3574 - val_accuracy: 0.8433
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0990 - accuracy: 0.9735 - val_loss: 0.4395 - val_accuracy: 0.8315
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0257 - accuracy: 0.9959 - val_loss: 0.5139 - val_accuracy: 0.8287
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0063 - accuracy: 0.9996 - val_loss: 0.5780 - val_accuracy: 0.8308
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.6346 - val_accuracy: 0.8303
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 9.1913e-04 - accuracy: 1.0000 - val_loss: 0.6736 - val_accuracy: 0.8309
Ep

In [49]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [51]:
#downloading the files for visualising word embeddings
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n") 
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [53]:
#this block will download the vectors file which contains coefficients of the word vectors in the vecs.tsv file
#meta.tsv file will contain the word itself
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
#testing with own sentence
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
padded = pad_sequences(sequence, maxlen=max_length, truncating=trunc_type)
#print(padded)
x = np.round(model.predict(padded))
print(x)

[[1.]]
